<a href="https://colab.research.google.com/github/AlexKitano/Portfolio/blob/main/Portfolio_Regressao_Multipla_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
from pyspark.sql.functions import monotonically_increasing_id, lit, col # Import necessary functions

SparkSession

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
.master("local[*]")\
.appName("Regressão com Spark")\
.getOrCreate()

spark

Carregamento dos dados

In [ ]:
dados = spark.read.csv("/content/df_test.csv", header = True)

In [ ]:
dados.show(truncate = False)

In [ ]:
# Criar coluna ID

from pyspark.sql.functions import monotonically_increasing_id

dados = dados.withColumn("ID", monotonically_increasing_id())

dados.show(truncate = False)

Seleção de Features

In [ ]:
dataset = dados\
.select("ID", "bedrooms", "grade", "has_basement", "living_in_m2", "renovated", "nice_view", "perfect_condition", "real_bathrooms", "has_lavatory", "single_floor", "month", "quartile_zone", "price")\
.drop("date")

In [ ]:
dataset.show()

In [ ]:
dataset.count()

Tratamento de Dados

In [ ]:
dataset.printSchema()

In [ ]:
from pyspark.sql.types import IntegerType, DoubleType, LongType, BooleanType

In [ ]:
dataset\
  .withColumn('grade', dataset['grade'].cast(IntegerType()))\
  .withColumn('bedrooms', dataset['bedrooms'].cast(IntegerType()))\
  .withColumn('has_basement', dataset['has_basement'].cast(BooleanType()))\
  .withColumn('living_in_m2', dataset['living_in_m2'].cast(DoubleType()))\
  .withColumn('renovated', dataset['renovated'].cast(BooleanType()))\
  .withColumn('nice_view', dataset['nice_view'].cast(BooleanType()))\
  .withColumn('perfect_condition', dataset['perfect_condition'].cast(BooleanType()))\
  .withColumn('real_bathrooms', dataset['real_bathrooms'].cast(IntegerType()))\
  .withColumn('has_lavatory', dataset['has_lavatory'].cast(BooleanType()))\
  .withColumn('single_floor', dataset['single_floor'].cast( BooleanType()))\
  .withColumn('month', dataset['month'].cast(IntegerType()))\
  .withColumn('quartile_zone', dataset['quartile_zone'].cast(IntegerType()))\
  .withColumn('price', dataset['price'].cast(LongType()))\
  .printSchema()

In [ ]:
dataset = dataset\
  .withColumn('grade', dataset['grade'].cast(IntegerType()))\
  .withColumn('bedrooms', dataset['bedrooms'].cast(IntegerType()))\
  .withColumn('has_basement', dataset['has_basement'].cast(BooleanType()))\
  .withColumn('living_in_m2', dataset['living_in_m2'].cast(DoubleType()))\
  .withColumn('renovated', dataset['renovated'].cast(BooleanType()))\
  .withColumn('nice_view', dataset['nice_view'].cast(BooleanType()))\
  .withColumn('perfect_condition', dataset['perfect_condition'].cast(BooleanType()))\
  .withColumn('real_bathrooms', dataset['real_bathrooms'].cast(IntegerType()))\
  .withColumn('has_lavatory', dataset['has_lavatory'].cast(BooleanType()))\
  .withColumn('single_floor', dataset['single_floor'].cast( BooleanType()))\
  .withColumn('month', dataset['month'].cast(IntegerType()))\
  .withColumn('quartile_zone', dataset['quartile_zone'].cast(IntegerType()))\
  .withColumn('price', dataset['price'].cast(LongType()))

In [ ]:
dataset.show()

In [ ]:
dataset\
.select('bedrooms')\
.groupby('bedrooms')\
.count()\
.show()

In [ ]:
dataset\
.select('grade')\
.groupby('grade')\
.count()\
.show()

In [ ]:
dataset\
.select('has_basement')\
.groupby('has_basement')\
.count()\
.show()

In [ ]:
dataset\
.select('renovated')\
.groupby('renovated')\
.count()\
.show()

In [ ]:
dataset\
.select('nice_view')\
.groupby('nice_view')\
.count()\
.show()

In [ ]:
dataset\
.select('perfect_condition')\
.groupby('perfect_condition')\
.count()\
.show()

In [ ]:
dataset\
.select('real_bathrooms')\
.groupby('real_bathrooms')\
.count()\
.show()

In [ ]:
dataset\
.select('has_lavatory')\
.groupby('has_lavatory')\
.count()\
.show()

In [ ]:
dataset\
.select('month')\
.groupby('month')\
.count()\
.show()

In [ ]:
dataset\
.select('quartile_zone')\
.groupby('quartile_zone')\
.count()\
.show()

Tratamento para dados faltantes

In [ ]:
from pyspark.sql import functions as f

In [ ]:
dataset.show()

Contagem de valores faltantes

In [ ]:
from pyspark.sql import functions as f
from pyspark.sql import types # Import the types module

dataset\
.select([f.count(f.when((f.col(c).cast("double").isNull()) | (f.isnan(f.col(c).cast("double"))), True)).alias(c) if dataset.schema[c].dataType in [types.DoubleType(), types.FloatType()]
         else f.count(f.when(f.col(c).isNull(), True)).alias(c)
         for c in dataset.columns])\
.show()

Regressão Linear

Variáveis Dummy

Preparando dados para regressão

In [ ]:
dataset.show()

In [ ]:
# Renomear coluna bedrooms pra 1 quarto, 2 quartos e 3 quartos

from pyspark.sql.functions import when

# Criar uma nova coluna com base nos valores de 'bedrooms'
dataset = dataset.withColumn(
    "bedrooms_category",
     when(dataset["bedrooms"] == 1, "1 bedroom")
    .when(dataset["bedrooms"] == 2, "2 bedrooms")
    .when(dataset["bedrooms"] == 3, "3 bedrooms")
    .otherwise("Outros")  # Define um valor padrão para outros números de bedrooms
)

In [ ]:
dataset = dataset.drop("bedrooms")

In [ ]:
dataset\
  .groupBy("ID")\
  .pivot("bedrooms_category")\
  .agg(f.lit(1))\
  .na\
  .fill(0)\
  .show()

In [ ]:
bedrooms_category = dataset\
                    .groupBy("ID")\
                    .pivot("bedrooms_category")\
                    .agg(f.lit(1))\
                    .na\
                    .fill(0)

In [ ]:
dataset = dataset.drop("bedrooms_category")

In [ ]:
# Renomear coluna grade pra 1 grade, 2 grades e 3 grades

from pyspark.sql.functions import when

# Criar uma nova coluna com base nos valores de 'bedrooms'
dataset = dataset.withColumn(
    "grades_category",
     when(dataset["grade"] == 1, "1 grade")
    .when(dataset["grade"] == 2, "2 grades")
    .when(dataset["grade"] == 3, "3 grades")
    .when(dataset["grade"] == 4, "4 grades")
    .when(dataset["grade"] == 5, "5 grades")

    .otherwise("Outros")  # Define um valor padrão para outros números de grades
)

In [ ]:
dataset = dataset.drop("grade")

In [ ]:
dataset\
  .groupBy("ID")\
  .pivot("grades_category")\
  .agg(f.lit(1))\
  .na\
  .fill(0)\
  .show()

In [ ]:
grade_category = dataset\
                  .groupBy("ID")\
                  .pivot("grades_category")\
                  .agg(f.lit(1))\
                  .na\
                  .fill(0)


In [ ]:
dataset = dataset.drop("grades_category")


In [ ]:
from pyspark.sql.functions import when

dataset = dataset.withColumn(
    "has_basement",
    when(dataset["has_basement"], 1).otherwise(0)
)

In [ ]:
dataset = dataset.withColumn(
    "renovated",
    when(dataset["renovated"], 1).otherwise(0)
)

In [ ]:
dataset = dataset.withColumn(
    "nice_view",
    when(dataset["nice_view"], 1).otherwise(0)
)

In [ ]:
dataset = dataset.withColumn(
    "perfect_condition",
    when(dataset["perfect_condition"], 1).otherwise(0)
)

In [ ]:
# Renomear coluna real_bedrooms pra 1 real bathroom, 2 real bathrooms e 3 real bathrooms

from pyspark.sql.functions import when

# Criar uma nova coluna com base nos valores de 'real_bedrooms'
dataset = dataset.withColumn(
    "real_bathrooms_category",
     when(dataset["real_bathrooms"] == 1, "1 real bathroom")
    .when(dataset["real_bathrooms"] == 2, "2 real bathrooms")
    .when(dataset["real_bathrooms"] == 3, "3 real bathrooms")

    .otherwise("Outros")  # Define um valor padrão para outros números de grades
)

In [ ]:
dataset = dataset.drop("real_bathrooms")

In [ ]:
dataset\
  .groupBy("ID")\
  .pivot("real_bathrooms_category")\
  .agg(f.lit(1))\
  .na\
  .fill(0)\
  .show()

In [ ]:
real_bathrooms_category = dataset\
                          .groupBy("ID")\
                          .pivot("real_bathrooms_category")\
                          .agg(f.lit(1))\
                          .na\
                          .fill(0)

In [ ]:
dataset = dataset.drop("real_bathrooms_category")

In [ ]:
dataset = dataset.withColumn(
    "single_floor",
    when(dataset["single_floor"], 1).otherwise(0)
)

In [ ]:
dataset = dataset.withColumn(
    "has_lavatory",
    when(dataset["has_lavatory"], 1).otherwise(0)
)

In [ ]:
# Renomear coluna month pra Janeiro, Fevereiro, Março, Abril, Maio, Junho,
# Julho, Agosto, Setembro, Outubro, Novembro e Dezembro.

from pyspark.sql.functions import when

# Criar uma nova coluna com base nos valores de 'real_bedrooms'
dataset = dataset.withColumn(
    "month_category",
     when(dataset["month"] == 1, "Janeiro")
    .when(dataset["month"] == 2, "Fevereiro")
    .when(dataset["month"] == 3, "Março")
    .when(dataset["month"] == 4, "Abril")
    .when(dataset["month"] == 5, "Maio")
    .when(dataset["month"] == 6, "Junho")
    .when(dataset["month"] == 7, "Julho")
    .when(dataset["month"] == 8, "Agosto")
    .when(dataset["month"] == 9, "Setembro")
    .when(dataset["month"] == 10, "Outubro")
    .when(dataset["month"] == 11, "Novembro")
    .when(dataset["month"] == 12, "Dezembro")

    .otherwise("Outros")  # Define um valor padrão para outros números de grades
)

In [ ]:
dataset = dataset.drop("month")

In [ ]:
dataset\
  .groupBy("ID")\
  .pivot("month_category")\
  .agg(f.lit(1))\
  .na\
  .fill(0)\
  .show()

In [ ]:
month_category = dataset\
                .groupBy("ID")\
                .pivot("month_category")\
                .agg(f.lit(1))\
                .na\
                .fill(0)

In [ ]:
dataset = dataset.drop("month_category")

In [ ]:
# Renomear coluna real_bedrooms pra 1 real bathroom, 2 real bathrooms e 3 real bathrooms

from pyspark.sql.functions import when

# Criar uma nova coluna com base nos valores de 'real_bedrooms'
dataset = dataset.withColumn(
    "quartile_zone_category",
     when(dataset["quartile_zone"] == 1, "1 quartile zone")
    .when(dataset["quartile_zone"] == 2, "2 quartiles zones")
    .when(dataset["quartile_zone"] == 3, "3 quartiles zones")
    .when(dataset["quartile_zone"] == 4, "4 quartiles zones")

    .otherwise("Outros")  # Define um valor padrão para outros números de grades
)

In [ ]:
dataset = dataset.drop("quartile_zone")

In [ ]:
dataset\
  .groupBy("ID")\
  .pivot("quartile_zone_category")\
  .agg(f.lit(1))\
  .na\
  .fill(0)\
  .show()

In [ ]:
quartile_zone_category = dataset\
                        .groupBy("ID")\
                        .pivot("quartile_zone_category")\
                        .agg(f.lit(1))\
                        .na\
                        .fill(0)

In [ ]:
dataset = dataset.drop("quartile_zone_category")

In [ ]:
dataset = dataset\
  .join(bedrooms_category, on = "ID", how = "inner")\
  .join(grade_category, on = "ID", how = "inner")\
  .join(real_bathrooms_category, on = "ID", how = "inner")\
  .join(month_category, on = "ID", how = "inner")\
  .join(quartile_zone_category, on = "ID", how = "inner")

In [ ]:
dataset.show()

Vetorização dos dados

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
dataset = dataset.withColumnRenamed("price", "label")

In [ ]:
X = ['has_basement',
     'living_in_m2',
     'renovated',
     'nice_view',
     'perfect_condition',
     'has_lavatory',
     'single_floor',
     '1 bedroom',
     '2 bedrooms',
     '3 bedrooms',
     '1 grade',
     '2 grades',
     '3 grades',
     '4 grades',
     '5 grades',
     '1 real bathroom',
     '2 real bathrooms',
     '3 real bathrooms',
     'Janeiro',
     'Fevereiro',
     'Março',
     'Abril',
     'Maio',
     'Junho',
     'Julho',
     'Agosto',
     'Setembro',
     'Outubro',
     'Novembro',
     'Dezembro',
     '1 quartile zone',
     '2 quartiles zones',
     '3 quartiles zones',
     '4 quartiles zones'
]

In [ ]:
assembler = VectorAssembler(inputCols = X, outputCol = "features")

In [ ]:
dataset_prep = assembler.transform(dataset).select("features", "label")

In [ ]:
dataset_prep.show(10, truncate = False)

Exploração de dados

In [ ]:
from pyspark.ml.stat import Correlation
import pandas as pd

In [ ]:
correlacao = Correlation.corr(dataset_prep, "features").collect()

In [ ]:
correlacao

In [ ]:
correlacao = Correlation.corr(dataset_prep, "features").collect()[0][0]

In [ ]:
correlacao

In [ ]:
import numpy as np
from pyspark.ml.linalg import DenseMatrix

correlacao_np = correlacao
# Converte matriz densa para numpy array
correlacao_np = correlacao_np.toArray()
correlacao_list = correlacao_np.tolist()  # Converte para lista

In [ ]:
import pandas as pd
import numpy as np

# # Remodelar a matriz para corresponder ao formato esperado da matriz de correlação
num_features = len(X)

# Converte matriz densa para numpy array
correlacao_np = correlacao.toArray()
correlacao_np = correlacao_np.reshape((num_features, num_features))  # Remodelar para 34x34

dataframe_correlacao = pd.DataFrame(correlacao_np, columns=X, index=X)

In [ ]:
dataframe_correlacao

Mapa de calor

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize = (12, 10))
paleta = sns.color_palette("light:salmon", as_cmap = True)
sns.heatmap(dataframe_correlacao.round(1), annot = True, cmap = paleta)

Ajuste e Previsão

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
treino, teste = dataset_prep.randomSplit([0.7, 0.3], seed = 101)

In [ ]:
treino.count()

In [ ]:
teste.count()

In [ ]:
lr = LinearRegression()

In [ ]:
modelo_lr = lr.fit(treino)

In [ ]:
previsoes_lr_treino = modelo_lr.transform(treino)

In [ ]:
previsoes_lr_treino.show()

Métricas

In [ ]:
resumo_treino = modelo_lr.summary

In [ ]:
resumo_treino.r2

In [ ]:
resumo_treino.rootMeanSquaredError

In [ ]:
resumo_teste = modelo_lr.evaluate(teste)

In [ ]:
resumo_teste.r2

In [ ]:
resumo_teste.rootMeanSquaredError

Tabela Resumo Regressão Linear

In [ ]:
print("Linear Regression")
print("="*30)
print("Dados de Treino")
print("="*30)
print("R²: %f" % resumo_treino.r2)
print("RMSE: %f" % resumo_treino.rootMeanSquaredError)
print("")
print("="*30)
print("Dados de Teste")
print("="*30)
print("R²: %f" % resumo_teste.r2)
print("RMSE: %f" % resumo_teste.rootMeanSquaredError)

Prevendo resultados

In [ ]:
dataset.show(15)

In [ ]:
X

In [ ]:
novo_imovel = [{
 'has_basement':2,
 'living_in_m2':103,
 'renovated':1,
 'nice_view':1,
 'perfect_condition':1,
 'has_lavatory':1,
 'single_floor':1 ,
 '1 bedroom':0 ,
 '2 bedrooms':0,
 '3 bedrooms':1,
 '1 grade':0,
 '2 grades':0,
 '3 grades':1,
 '4 grades':0,
 '5 grades':0,
 '1 real bathroom':1,
 '2 real bathrooms':0,
 '3 real bathrooms':0,
 'Janeiro':1,
 'Fevereiro':0,
 'Março':0,
 'Abril':0,
 'Maio':0,
 'Junho':0,
 'Julho':0,
 'Agosto':0,
 'Setembro':0,
 'Outubro':0,
 'Novembro':0,
 'Dezembro':0,
 '1 quartile zone':0,
 '2 quartiles zones':0,
 '3 quartiles zones':1,
 '4 quartiles zones':0,
 'label':0}]

In [ ]:
meu_imovel = spark.createDataFrame(novo_imovel)

In [ ]:
meu_imovel.show()

In [ ]:
assembler = VectorAssembler(inputCols = X, outputCol = "features")

In [ ]:
meu_lar_vetorizado = assembler.transform(meu_imovel).select("features", "label")

In [ ]:
meu_lar_vetorizado.show()

In [ ]:
# Prever price do meu_lar_vetorizado

previsao = modelo_lr.transform(meu_lar_vetorizado)
print(f"O preço previsto para o imóvel é de: {previsao.select('prediction').collect()[0][0]}")
